In [34]:
import pandas as pd
import numpy as np

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.image as mpimg

# Préparation des données
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Modèles
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance

# Option d'affchage
pd.set_option('display.max_columns', None)

In [35]:
df0 = pd.read_csv('/Users/lilian/Desktop/hackathon2021/data/train.csv')

/Users/lilian/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning:

Columns (13,32) have mixed types.Specify dtype option on import or set low_memory=False.



In [43]:
df = df0.copy().sample(10**6)

In [44]:
df.sample(10)

,date,store_id,parent_chain_id,store_country,country_iso_code,region_id,store_region,store_segment,store_activity_name,items_first_enabled_date,store_first_saving_date,store_last_saving_date,item_id,item_name,before_price,currency_code,pickup_start,pickup_end,total_supply,declared_supply,manual_added_supply,manual_removed_supply,meals_saved,consumer_cancellation,store_cancellation,item_price,meals_refunded,rating_count,sum_rating_overall,item_view,no_unique_consumers,is_enabled,Département,target
4103836,2020-02-29,36711,3267.0,France,FR,FRJ2,Midi-Pyrénées,Buffet,Commercial,2019-07-17,2019-07-17,2020-10-28,36874,Café,9.0,EUR,2020-02-29 18:00:00.000000,2020-02-29 19:00:00.000000,2,2,0,0,2,0,0,3.00,0,1,5,10,2,True,31,0.0
1473198,2019-01-30,29458,4669.0,France,FR,FR10,Ile-De-France,Fast Food / Take away restaurant,Unknown,2018-03-24,2018-03-24,2021-03-12,29621,NaN,12.0,EUR,2019-01-30 14:00:00.000000,2019-01-30 15:00:00.000000,5,3,2,0,3,0,0,4.00,0,1,3,0,2,True,95,0.0
2549502,2020-01-11,32384,1214.0,France,FR,FRF2,Champagne-Ardenne,Bakery,Commercial,2018-10-19,2018-10-19,2021-02-19,32547,NaN,12.0,EUR,2020-01-11 17:30:00.000000,2020-01-11 18:00:00.000000,4,0,4,0,4,0,0,3.99,0,3,15,29,4,True,51,0.0
4884659,2020-04-26,45240,4564.0,France,FR,FRI1,Aquitaine,Supermarket Medium,Commercial,2019-09-26,2019-10-02,2019-10-18,46685,NaN,12.0,EUR,NaN,NaN,0,0,0,0,0,0,0,3.99,0,0,0,0,0,True,33,1.0
2481758,2020-04-07,32210,3314.0,France,FR,FR10,Ile-De-France,Supermarket Medium,Commercial,2018-10-22,2018-10-22,2020-08-05,32373,NaN,12.0,EUR,NaN,NaN,0,0,0,0,0,0,0,4.00,0,0,0,5,0,True,94,0.0
132866,2019-07-12,25242,NaN,France,FR,FRJ2,Midi-Pyrénées,Supermarket Small,Commercial,2019-07-11,2019-07-11,2019-07-20,25357,NaN,10.0,EUR,2019-07-12 07:30:00.000000,2019-07-12 10:30:00.000000,3,1,2,0,3,0,0,2.99,0,0,0,0,2,True,81,0.0
861485,2020-03-10,27798,NaN,France,FR,FRI1,Aquitaine,Bakery,Unknown,2017-10-13,2017-12-02,2021-03-15,27962,NaN,12.0,EUR,2020-03-10 18:50:00.000000,2020-03-10 19:00:00.000000,8,2,6,0,8,0,0,4.00,0,3,13,85,8,True,33,0.0
3795412,2019-11-20,35773,1953.0,France,FR,FR10,Ile-De-France,Fast Food / Take away restaurant,Commercial,2019-03-31,2019-03-31,2019-11-30,35936,NaN,18.0,EUR,2019-11-20 15:00:00.000000,2019-11-20 19:00:00.000000,6,0,10,-4,6,0,0,5.99,0,2,6,52,5,True,78,0.0
1211932,2019-12-03,28766,5041.0,France,FR,FRI1,Aquitaine,Bakery,Commercial,2018-01-31,2018-01-31,2021-03-15,28929,NaN,10.0,EUR,2019-12-03 18:45:00.000000,2019-12-03 19:00:00.000000,10,10,0,0,10,0,0,3.00,0,3,15,107,7,True,33,0.0
502898,2019-02-17,26735,NaN,France,FR,FR10,Ile-De-France,Traditional Restaurant,Commercial,2017-05-20,NaN,NaN,26902,NaN,NaN,EUR,NaN,NaN,0,0,0,0,0,0,0,6.00,0,0,0,0,0,True,75,1.0


### Construction de variables intéressantes à partir des préexistantes

In [45]:
# Note moyenne

def compute_note(overall, count):
    try:
        overall / count
    except:
        return np.nan

df['note_moyenne'] = df.apply(lambda row : compute_note(row.sum_rating_overall, row.rating_count) , axis = 1)

In [46]:
# Lifetime

col_date = ['date', 'items_first_enabled_date', 'store_last_saving_date', 'store_first_saving_date',\
            'pickup_start', 'pickup_end']
for col in col_date :
    df[col]= pd.to_datetime(df[col])

df['lifetime'] = df['store_last_saving_date'] - df['store_first_saving_date']



In [47]:
# Reduction

df['reduction'] = 1 - df['item_price'] / df['before_price']

In [48]:
# Temps d'ouverture

df['temps_ouverture'] = df['pickup_end'] - df['pickup_start']

In [49]:
# Heure de début d'ouverture

df['heure_debut_ouverture'] = df.apply(lambda row : row.pickup_start.hour, axis = 1)


In [50]:
# Efficacité

df['efficacite'] = df['meals_saved'] - df['total_supply']

In [51]:
# Franchise

df['franchise'] = df.apply(lambda row : int(row.parent_chain_id > 0), axis = 1)

### Création de nouvelles variables

In [79]:
# Variance du nombre de vente pour un shop

def compute_variance(date, store_id, variable):
    df_short = df[df['store_id'] == store_id]
    df_short = df_short[df_short['date'] < date]
    return df_short[variable].var()

df['variance_ventes'] = df.apply(lambda row : compute_variance(row.date, row.store_id, 'meals_saved'), axis = 1)

KeyboardInterrupt: 

In [ ]:
# Baisse du nombre de vente dans le mois précedent

def compute_baisse_vente(date, store_id, variable):
    df_short = df[df['store_id'] == store_id]
    df_short = df_short[df_short['date'] < date]
    serie = df_short[variable]
    
    try:
        b = serie[:-7] / 
    return 

df['variance_ventes'] = df.apply(lambda row : compute_variance(row.date, row.store_id, 'meals_saved'), axis = 1)

In [ ]:
# Augmentation de la réduction durant la semaine précedente

In [ ]:
# Variance de la durée d'ouverture

In [ ]:
# Baisse de la note moyenne durant la semaine précedente

In [ ]:
# Augmentation du nombre d'invendus durant le mois précédent

### Suppression des variables non intéressantes

In [ ]:
df = df.drop(columns = ['store_country', 'country_'])
df.sample(10)

## Recodage de la variable objectif